# Introdução ao Processamento de Linguagem Natural usando Python

# Antes de começar!

### Dicas!

Para instalar novos pacotes:

 - pip install nome_do_pacote

OU 

 - conda install nome_do_pacote 


### Pacotes necessarios: 
    - conda install -c conda-forge wordcloud 
    - pip install nltk
    - pip install pandas
    - pip install spacy
    

## Atenção!

Os nomes das lojas foram ocultados para mantermos a segurança da loja.

A empresa foi substituído por códigos para representar cada uma individualmente.

Dentro do corpo da reclamação, o nome da loja foi substituído  por "LOJA"

Nomes e emails também foram removidos.

# Atenção!

Antes de começarmos é necessario baixar a lista de stopwords, sentence tokenization e o corpus do NLTK

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('machado')

# 1. Exemplo -  Corpus

In [ ]:
from nltk.corpus import machado

# Verificando o conjunto de textos contido no Corpus Machado
print(machado.fileids())

# Cada arquivo corresponde a uma das obras de Machado de Assis. 

In [ ]:
# Para acessar o texto como uma lista de 200 caracteres..
raw_machado_text = machado.raw('romance/marm05.txt') # explore outros arquivos
raw_machado_text[10000:10200]

In [ ]:
# Porem, a forma acima não é uma maneira muito útil de trabalhar com um texto. 
# Geralmente pensamos em um texto como uma sequência de palavras e pontuação, não em caracteres..
text1 = machado.words('romance/marm05.txt')
text1

In [ ]:
# Checando o tamanho do romance marm05.txt
print('Quantidade de palavras {}'.format(len(text1)))
# Checando o tamanho de palavras unicas do texto
print('Quantidade de palavras unicas {}'.format(len(set(text1))))

In [ ]:
# Sua vez: O que mais podemos fazer?
# Liste aqui suas ideia!

http://www.nltk.org/howto/portuguese_en.html

# Dados Reclame Aqui

## 1. Importando os Dados

### Para executar na sua maquina local

In [ ]:
# Descomente essa celula

# # Importando Pandas
# import pandas as pd

# # Vamos agora importar os dados que vamos trabalhar!
# reclamacoes = pd.read_csv('reclamacoes.csv', sep=';')

# print(reclamacoes.shape)

# reclamacoes.head()

### Para executar no Google Colab 

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import io
data = io.BytesIO(uploaded['reclamacoes.csv'])    

import pandas as pd   
reclamacoes = pd.read_csv(data, sep=';')

reclamacoes.head()

In [ ]:
# Verificando se tem dados faltantes
reclamacoes.isna().sum()

## 2.Pré-Processamento

### 2.1 Quebrando a coluna data_reclamacao em dia e hora

In [ ]:
# Vamos separar as informações

print('Como era antes:')
print(reclamacoes['data_reclamacao'][:3])

# Separando...
reclamacoes['data'] = reclamacoes['data_reclamacao'].str.split('às',expand=True)[0]
# print(reclamacoes['data_reclamacao'].str.split('às',expand=True))
print('\n')
print('Como ficou agora:')
print(reclamacoes['data'][:3])

In [ ]:
# Sua vez: Faça a mesma coisa para a informação hora

# Você consegue identificar algo que está faltando ao utilizar essa nossa técnica?
# Dica: Utilize reclamacoes.data[0]

In [ ]:
# Como resolver o problema acima?
# Sua solução:

# Verifique se a sua solução funcionou:


### 2.2 Quebrando a coluna local em Cidade e Estado

In [ ]:
# Sua vez: Faça a mesma coisa para que você fez acima para a variavel local e estado
# não se esqueça do problema que resolvemos acima!


In [ ]:
# Visualizando as alterações que fizemos
reclamacoes[['data_reclamacao', 'data', 'hora', 'local', 'cidade', 'estado']].head()

### 2.3 Alterar tipo de colunas

In [ ]:
# Verificando o tipo de dados 
reclamacoes.info()

In [ ]:
# Colunas que são categoricas
reclamacoes['teve_resposta'] = reclamacoes['teve_resposta'].astype('category')

# Colunas que são datetime
reclamacoes['data'] = pd.to_datetime(reclamacoes['data'])

reclamacoes.info()

### 2.4 LowerCase

In [ ]:
print('Antes:')
print(reclamacoes['corpo_reclamacao'].head())

In [ ]:
# Aplicando Lower Case
reclamacoes['corpo_reclamacao_lower'] = [str(token).lower() for token in reclamacoes['corpo_reclamacao']]

print('Depois:')
reclamacoes.corpo_reclamacao_lower.head()

### 2.5 Tokenização

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize

# Vamos dividir um exemplo texto em tokens/palavras:
tokens = word_tokenize('A história do NLP começou na década de 1950, com Alan Turing'.lower())

print(tokens)

print('\nQuantidade de Tokens: {}'.format(len(tokens)))

In [ ]:
# Agora vamos dividir um exemplo texto em sentenças:
sentencas = sent_tokenize('O Processamento de Linguagem Natural (PLN) é a subárea da Inteligência Artificial (IA) que estuda a capacidade e as limitações de uma máquina em entender a linguagem dos seres humanos. O objetivo do PLN é fornecer aos computadores a capacidade de entender e compor textos. “Entender” um texto significa reconhecer o contexto, fazer análise sintática, semântica, léxica e morfológica, criar resumos, extrair informação, interpretar os sentidos, analisar sentimentos e até aprender conceitos com os textos processados.') 

# Visualizando cada sentença
print(sentencas[0])
print()
print(sentencas[1])
print()
print(sentencas[2])

print('\nQuantidade de Sentenças: {}'.format(len(sentencas)))

In [ ]:
# Sua vez: Separe os nossos textos em tokens e coloque em uma nova coluna chamada corpo_reclamacao_tokens


In [ ]:
# Como ficou nosso dataset até agora
reclamacoes.head()

### 2.6 Remoção das StopWords

In [ ]:
from nltk.corpus import stopwords
from string import punctuation
import re # Regex

#  Regex para encontrar pontuações dentro do texto
other_punctuation_reg = r'(\.{2,}|-+|—+|\|+)'

other_punctuation_finds = []

# Percorre a lista de reclamções e identica as pontuações
for text in reclamacoes.corpo_reclamacao_lower:
    # encontra as pontuações
    p = re.search(other_punctuation_reg, text)
    if p:
        other_punctuation_finds.append(p.group())

# Gera lista com stopwords em Português  
portuguese_stopswords = set(stopwords.words('portuguese') + list(punctuation) + other_punctuation_finds)

# Adicionando outras palavras na nossa lista de StopWords
portuguese_stopswords.add('é')

portuguese_stopswords


O comando stopwords.words("portuguese") lista todas as stopwords em português; 

OU

Para visualizar a lista de StopWords:

Windows:
C:\Users\SEU_USUARIO\AppData\Roaming\nltk_data\corpora\stopwords

In [ ]:
# Verificando como era antes
print('Antes de remover as StopWords:')
reclamacoes.corpo_reclamacao_lower[0] # visualizando somente a primeira reclamação

In [ ]:
# Exemplo - Removendo StopWords da primeira reclamação
palavras_sem_stopwords = [token for token in reclamacoes['corpo_reclamacao_tokens'][0] if token not in portuguese_stopswords]
palavras_sem_stopwords

In [ ]:
# Função que remove as stopwords
def remove_stopwords(text, portuguese_stopswords):
    print('Tamanho do texto original {}'.format(len(word_tokenize(text))))
    # Remove as stopwords
    msgTokens_sem_stopwords = [token.lower() for token in word_tokenize(text) if token not in portuguese_stopswords]
    print('Tamanho do texto sem stopwords {}'.format(len(msgTokens_sem_stopwords)))
    print('Foram removidas {} stopwords'.format(len(word_tokenize(text))-len(msgTokens_sem_stopwords)))
    aux_msgTokens = " ".join(str(msg) for msg in msgTokens_sem_stopwords)
    return aux_msgTokens

In [ ]:
# Removendo StopWords de todas as reclamações
   
# Percorre a lista de reclamações e cria uma coluna nova com o texto sem stopWords
for idx,text in enumerate(reclamacoes.corpo_reclamacao_lower):
    print('Removendo StopWords do index {}'.format(idx))
    reclamacoes.at[idx, 'corpo_reclamacao_semStopWords'] = remove_stopwords(text, portuguese_stopswords)
    print('---'*20)

In [ ]:
# Removendo StopWords de todas as Tags

# Percorre a lista de reclamações e cria uma coluna nova com o tags sem stop Words
reclamacoes.tags = reclamacoes.tags.astype(str)
for idx,text in enumerate(reclamacoes.tags):
    print('Removendo StopWords do index {}'.format(idx))
    reclamacoes.at[idx, 'Tags_semStopWords'] = remove_stopwords(text, portuguese_stopswords)
    print('---'*20)


In [ ]:
reclamacoes.head()

### 2.7 Normalização das palavras - Stemming e Lemmatization 

Na língua portuguesa temos diferentes palavras flexionadas em gênero, número ou grau, além de alguns tempos verbais distintos.

*Reclamar – Reclamação*

*Problema – Problemão*

A normalização de palavras pode ser entendida como a simplificação ou a radicalização de palavras.

E existem duas técnicas mais conhecidas:

*Stemming* - O processo de stemming consiste em reduzir a palavra à sua raiz, não levando em consideração a classe gramatical da palavra.

*Lemmatization* - Consiste em aplicar técnicas para deflexionar as palavras (retira a conjugação verbal, caso seja um verbo, e altera os substantivos e os adjetivos para o singular masculino, de maneira a reduzir a palavra até sua forma de dicionário).

Ex.:

Menino: menino, menina, meninão

Ficar: ficar, ficou, fique



In [ ]:
nltk.download('rslp')
stemmer = nltk.stem.RSLPStemmer()

palavras = ['reclamação', 'reclamei', 'reclamando']

for w in palavras:
    print(stemmer.stem(w))
 

### Lemmatization

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
 
lemmatizer = WordNetLemmatizer()

Exemplo de lematização, porém não existe uma biblioteca em português apenas em inglês no momento.

In [ ]:
palavras = ['jumps', 'ladies', 'oranges']

for w in palavras:
    print(lemmatizer.lemmatize(w))
    

## 3.Análise dos dados

### 3.1 Analise Empresas

In [ ]:
# Gráficos
import matplotlib.pyplot as plt

%matplotlib inline
# Quantidade de reclamcoes por empresa
empresa = reclamacoes.groupby("empresa")

plt.figure(figsize=(10,5))
empresa.size().sort_values(ascending=False).plot.bar()
plt.title("Quantidade de reclamacões por empresa")
plt.xticks(rotation=0)
plt.xlabel("Empresas")
plt.ylabel("Número de reclamações")
plt.show()

### 3.2 Cidade e Estado

#### Cidades

In [ ]:
# Vendo quando locais recebem vem as reclamações
cidades = reclamacoes.groupby("cidade")

plt.figure(figsize=(13,5))
cidades.size().sort_values(ascending=False).plot.bar()
plt.xticks(rotation=90)
plt.xlabel("Cidades")
plt.ylabel("Numero de reclamações")
plt.show()

#### Estados

In [ ]:
# Vendo quando locais recebem vem as reclamações
estados = reclamacoes.groupby("estado")

plt.figure(figsize=(13,5))
estados.size().sort_values(ascending=False).plot.bar()
plt.xticks(rotation=90)
plt.xlabel("Estados")
plt.ylabel("Numero de reclamações")
plt.show()

### 3.3 Frequencia de palavras de todas as reclamações 

In [ ]:
# NLTK
from nltk.probability import FreqDist

#função para ver as palavras mais frequente de todas as reclamações 

def freq_Words(text, number,titulo):
    words = word_tokenize(text)
    fdist = FreqDist(word for word in words)
    XY = fdist.items()
    XY = sorted(XY, key=lambda pair: pair[1], reverse=True)
    limit = number
    X = [x for (x,y) in XY[:limit]]
    Y = [y for (x,y) in XY[:limit]]
    nX = range(len(X))
    plt.figure(figsize=(13,5))
    plt.title("Tokens mais frequentes das {}".format(titulo))
    plt.plot(nX, Y, label='contagens dos tokens', color = "red")
    plt.xticks(nX, X, rotation='vertical')
    plt.xticks(rotation='60')
    plt.xlabel('Tokens')
    plt.ylabel('Contagens')
    plt.grid()
    plt.legend(loc='upper right', fontsize='small')
    #plt.tight_layout()
    plt.show()
    return fdist

In [ ]:
#juntando todas as mensagens em um unico texto
lista_reclamacao = ' '.join(reclamacoes.corpo_reclamacao_semStopWords.tolist())
frequencia = freq_Words(lista_reclamacao,10,"reclamações")

### 3.4 Frequencia de palavras de todas as tags 

In [ ]:
lista_tags = ' '.join(reclamacoes.Tags_semStopWords.tolist())
frequencia = freq_Words(lista_tags,10,"tags")

### 3.5 Frequencia de Respostas 

In [ ]:
reclamacoes.teve_resposta = reclamacoes.teve_resposta.replace(0,"Não Respondido")
reclamacoes.teve_resposta = reclamacoes.teve_resposta.replace(1,"Respondido")      

respostas = reclamacoes.groupby("teve_resposta")
plt.figure(figsize=(10,5))
respostas.size().sort_values(ascending=False).plot.bar()
plt.title("Quantas reclamações foram respondidas?")
plt.xticks(rotation=0)
plt.xlabel("Respostas")
plt.ylabel("Numero de respostas")
plt.show()

## 4. Nuvem de palavras 

In [ ]:
#Juntando todos os textos 

# Textos com StopWords
todas_reclamacoes = " ".join(review for review in reclamacoes.corpo_reclamacao_lower)

# Texto sem StopWords
todas_reclamacoes_semStopWords = " ".join(review for review in reclamacoes.corpo_reclamacao_semStopWords)

print ("Temos {} palavras juntando todas as reclamações.".format(len(todas_reclamacoes_semStopWords)))

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import numpy as np
from PIL import Image

# World Cloud
# Exemplo: World cloud de todas reclamações sem stopwords

# Gerando a wordcloud
plt.figure(figsize=(10,5))
wordcloud = WordCloud(background_color="white").generate(todas_reclamacoes_semStopWords)

# Mostrando nuvem de palavras com o matplotlib 
plt.title('WordCloud sem StopWords', fontsize=20)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# World Cloud
# Exemplo: World cloud de todos os textos

# Gerando a wordcloud
plt.figure(figsize=(10,5))
wordcloud = WordCloud(background_color="white").generate(todas_reclamacoes)

#Mostrando nuvem de palavras com o matplotlib 
plt.title('WordCloud com StopWords', fontsize=20)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

Reparem na diferença na visualização. 

Na segunda, o resultado é muito menos util para um tomador de decisão que o primeiro.

Por exemplo, "problema", "pedido", "gerente", podem ser palavras a levantar um alerta dentor da empresa.

In [ ]:
# Exercicios: World cloud de uma unica empresa


### 4.1 Nuvem de palavras com Imagem

In [ ]:
# Selecionar a imagem que será utilizada como template
# OBS: A imagem precisa estar na mesma pasta que o notebook!

# Para executar na sua maquina
#py_mask = np.array(Image.open("girls.png")) 

# Para executar no Google Colab
uploaded = files.upload()
py_mask = np.array(Image.open(io.BytesIO(uploaded['girls.png'])))

# Cria a WordCloud passando a imagem selecionada acima como mascara
wc = WordCloud(background_color="white", max_words=10000, mask=py_mask,margin=1)

wc.generate(todas_reclamacoes_semStopWords)

# Salva a imagem na sua maquina :)
wc.to_file("girls_wordcloud.png")

plt.figure(figsize=[20,10])

image_colors = ImageColorGenerator(py_mask)
plt.imshow(wc.recolor(color_func=image_colors), interpolation='bilinear' )
plt.axis("off")


# BONUS

# Um pouco de Deep Learning!

Vamos agora entrar no mundo do Deep Learning!

Par isso vamos utilizar o Spacy, uma poderosa bilioteca de Processamento de Linguagem Natural e é focada em "entender" grandes quantidades de texto.

Para instalar descomente a celula abaixo:

<b>ATENÇÃO:</b> Para instalar o Spacy na sua maquina pessoal é necessario ter o Microsoft Visual C++ 14.0 (visual studio) instalado na maquina. No Colab ele já vem pré-instalado :)

In [ ]:
# Se já não tiver instalado o spacy...

!pip install spacy
# !python -m spacy download en
!python -m spacy download pt
!python -m spacy download pt_core_news_sm

## 1. Primeiros passos com o Spacy

In [ ]:
# Vamos usar o modelo pré-pronto que acabamos de baixar
import spacy

nlp = spacy.load('pt_core_news_sm') # Importando o modelo pré treinado de Deep Learning do Spacy em Português!

# Passando uma frase para o spacy
doc = nlp(u'Eai! Já ta sabendo tudo sobre NLP? A Camila conhece também, vou pedir pra ela me dar umas aulas')
# Obs: Repare no unicode na string...você deve usa-lo para que funcione corretamente.

# Tambem podemos Tokenizar o texto pelo Sapcy hehe
for token in doc:
    print(token.text)

## 2. Identificação de Entidades

A identificação de Entidade (em inglês Named Entity Recognition - NER) é a identificação de objetos/itens/coisas do mundo real, por exemplo, uma pessoa, um produto, um pais ou estado.

Com o Spacy é possivel se treinar seu proprio algoritmo de NER.

In [ ]:
# Entidades

doc = nlp(u"Apesar da Maria morar em São Paulo, ela me disse que seu sonho era morar em Nova York")

for ent in doc.ents:
    print('Texto: {}'.format(ent.text))
    print('Inicio: {}'.format(ent.start_char))
    print('Fim: {}'.format(ent.end_char))
    print('Entidade: {}'.format(ent.label_))
    print('----'*5)

Lista com os tipos de Entidade suportados/Identificados pelo Spacy: https://spacy.io/api/annotation#named-entities

In [ ]:
# Visualizando de uma forma mais bonita!

from spacy import displacy
from IPython.core.display import display, HTML

html = displacy.render(doc, style="ent")
display(HTML(html))

In [ ]:
# Sua vez!

# Exercicios: Aplique NER nas reclamações de uma empresa que você queira, quais entidades foram as mais retornadas?

## 3. Part-of-speech tagging - POS Tagging

### Análise de classes gramaticais 

Depois de tokenizar o texto é possivel identificar a classe gramatical de cada palavra/token (por exemplo, substantivo ou verbo)

Essa tecnica é bastante usada na geração automatica de legendas ou até mesmo na tradução automatica, pois com ela podemos entender o contexto da frase e inferir qual a proxima palavra é mais propensa a ser a correta

In [ ]:
doc = nlp(u'Apesar da Maria morar em São Paulo, ela me disse que seu sonho era morar em Nova York')

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

In [ ]:
html = displacy.render(doc, style="dep")
display(HTML(html))

# Fim :)